<a href="https://colab.research.google.com/github/VolkiTheDreamer/PythonRocks/blob/master/Machine%20Learning/Reinforcement%20Learning/Gym_CartPole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Starters

In [ ]:
!pip -q install ipython-autotime
%load_ext autotime

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Sıfırda manuel oluşturma

In [ ]:
import gymnasium as gym
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random
import pickle
import time
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

(Medium yazısına link, after publish)

Environemnet hakkındaki detay bilgilere [buradan](https://gymnasium.farama.org/environments/classic_control/cart_pole/) ulaşabilrisiniz.

Ben tensorflow kullancağım. Torch ile örnek uygulama göremk isterseniz
[burada](https://github.com/ritakurban/Practical-Data-Science/blob/master/DQL_CartPole.ipynb) detaylı bir anlatım sözkonusu, bakılabilir veya aşağıdaki benim kodları kendiniz torch'a çevirmek isteyebilirsiniz, bunla da uğraşmak istemezseniz ChatGPT veya benzeri bir araca dönüşüm sağlatabilirsiniz. Ancak ben sıfırdan sizin manuel yazmanızı tavsiye ederim.  

## Ortak Kullanılacak Train ve Test fonksiyonları

Bu fonksiyonları tüm yöntemlerde kullanacağız, o yüzden en başa aldım ama bu kısmı şimdi atlayıp öncelikle Pure DQL kısmına gelin. Orayı inceledikten sonra bunlara tekrar bakarsınız.

Öncelikle klasik Q-Learningde kurguladığımız eğitim kodunu kodlayalım. Dıştaki for döngümüz episodelar için olacak, içteki while döngümüz ise ilgili episode bitene kadar sürekli devam edecek.

In [ ]:
def train_model(env, agent, episodes = 100, print_episode=True):
  scores = []
  for e in tqdm(range(episodes)):
      state,_ = env.reset()
      if printInsideTraining:
        print(f"state before reshape: {state}, shape: {state.shape}")

      state = state.reshape(1,-1)
      if printInsideTraining:
        print(f"state after reshape: {state}, shape: {state.shape}")

      sayac = 0
      while True:
          if printInsideTraining:
            print(f"\nsayac:{sayac}")
          if agent is None:
            action = env.action_space.sample()
          else:
            action = agent.choose_action(state)
          if printInsideTraining:
              print(f"seçilen action:{action}")
          # bu action'a göre hareket edelim
          next_state, reward, terminated, truncated , _= env.step(action) #son parametre info olup gereksizdir
          done = terminated or truncated
          if printInsideTraining:
              print(f"nextstate before reshape:{next_state} ve shape'i: {next_state.shape}")

          next_state = next_state.reshape(1,-1)
          if printInsideTraining:
              print(f"nextstate after reshape:{next_state} ve shape'i: {next_state.shape}")
          if agent is not None:
            # yeni edindiğimiz deneyimi hafızaya atalım
            agent.add_experince_to_memory(state, action, reward, next_state, done)
          # state'i güncelleyelim
          state = next_state

          if agent is not None:
            # eğitimi tamamlayalım
            agent.replay_vectorized() #T4 GPU'da 100 episodes 13 dk, vectorize olmayan metod 9 saat sürdü
          if agent is not None:
            # epsilon'ı güncelleyelim
            if agent.epsilon > agent.epsilon_min:
                agent.epsilon *= agent.epsilon_decay
          #sayacı 1 artıralım
          sayac += 1
          if done:
              if print_episode:
                print(f"Episode: {e}, sayaç: {sayac}")
              scores.append(sayac)
              break
  plt.plot(scores)
  plt.axhline(np.mean(scores), color='red', linestyle='dashed')
  _, max_xlim = plt.xlim()
  plt.text(max_xlim*1.01, np.mean(scores)*0.99, f'Ortalama: {np.mean(scores):.1f}')
  plt.text(np.argmax(scores)*1.01, np.max(scores)*0.99, f'Max: {np.max(scores):.1f}')

  thresh = 500
  plt.axhline(thresh, color='blue')
  plt.text(max_xlim*1.01, thresh*0.99, f'Kazanma eşik noktası:{thresh}')
  plt.show();

In [ ]:
from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo
def test_model(env, trained_agent, n):
  #n kez oynasın

  scores=[]
  for i in tqdm(range(n)):
    state ,_= env.reset()
    state = state.reshape(1,-1)
    sayac = 0
    while True:
        if trained_agent is not None:
          action = trained_agent.choose_action(state)
        else:
          action = env.action_space.sample()
        next_state, reward, terminated, truncated ,_= env.step(action)
        done = terminated or truncated
        next_state = next_state.reshape(1,-1)
        state = next_state
        sayac += 1
        if done:
            break
    print(f"{i}. oyunda {sayac} hamle dayandı")
    scores.append(sayac)

  print(f"Ortalama dayanma süresi:{np.mean(scores)} hamle")
  print(f'Episode time taken: {env.time_queue}')
  print(f'Episode total rewards: {env.return_queue}')
  print(f'Episode lengths: {env.length_queue}')
  env.close()

In [ ]:
import base64
from IPython.display import HTML
import moviepy.editor as mpy

def show_video(video_path, slow_factor=None):
  if slow_factor is not None:
    clip = mpy.VideoFileClip(video_path)
    slow_motion_clip = clip.fx(mpy.vfx.speedx, 1/slow_factor)
    slow_motion_clip.write_videofile(f"{video_path[:-4]}_sw.mp4")
    video = open(f"{video_path[:-4]}_sw.mp4", 'rb').read()
  else:
    video = open(video_path, 'rb').read()
  encoded = base64.b64encode(video)
  return HTML(data='''<video alt="test" autoplay
                controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

def save_as_gif(video_path):
  clip = mpy.VideoFileClip(video_path)
  clip.write_gif(f"{video_path[:-4]}.gif")

## Random action stratejisi

In [ ]:
env = gym.make("CartPole-v1") #env'i yaratalım
printInsideTraining=False
train_model(env, agent = None, episodes=100, print_episode=False)

In [ ]:
# test
num_eval_episodes = 10
env = gym.make("CartPole-v1", render_mode="rgb_array")
env = RecordVideo(env, video_folder="cartpole-agent", name_prefix="random-test",
                  episode_trigger=lambda x: True)
env = RecordEpisodeStatistics(env, buffer_length=num_eval_episodes)
test_model(env, None, n=num_eval_episodes)

In [ ]:
show_video('cartpole-agent/random-test-episode-0.mp4', slow_factor=5)

## Pure DQL

In [ ]:
class DQLAgent:
    def __init__(self, env, loss="mse",batch_size=16,learning_rate=0.001,memory_size=1000):
        # bu class'tan bir obje yaratıldığında set edilecek değerler ve çalıştırılacak metodlar
        self.state_size = env.observation_space.shape[0] #4
        self.action_size = env.action_space.n #2
        self.gamma = 0.95
        self.learning_rate = learning_rate
        self.epsilon = 1  # explore
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.memory = deque(maxlen = memory_size) #replay meomry hacmi
        self.verbose = False
        self.batch_size= batch_size

        self.online_model = self.build_model(loss) # sadece Q networkünü oluşturuyoruz

    def save(self, file):
        pickle.dump(self, open(file, "wb"))

    def build_model(self, loss):
        model = Sequential()
        model.add(Dense(24, input_dim = self.state_size, activation = "relu")) #Ideal senaryoda node adedi tune edilebilir, biz şuan buna takılmayacağız, 48 sabit kalsın
        model.add(Dense(48, input_dim = self.state_size, activation = "relu")) #bu araya tercihe göre bir katman daha eklenebilir
        model.add(Dense(self.action_size, activation = "linear")) #output layer'ımızda action adedi kadar node olacak
        model.compile(loss = loss, optimizer = Adam(learning_rate = self.learning_rate))
        return model

    def add_experince_to_memory(self, state, action, reward, next_state, done):
        # deneyimleri hafızaya atıyoruz, sonra buradan random kayıt seçceğiz
        self.memory.append((state, action, reward, next_state, done))
        if self.verbose:
            print(f"replay memory hacmi:{len(self.memory)}")

    def choose_action(self, state):
        # acting: explore or exploit
        if random.uniform(0,1) <= self.epsilon:
            return env.action_space.sample()
        else:
            act_values = self.online_model.predict(state, verbose=0)
            return np.argmax(act_values[0])

    def _replay(self): #*********************************************************DİKKAT: Kullanılmayacak, alttaki açıklamayı okuyun******************************************************
        # öğrenemyi sağlayan metoddur. adını train veya learn olarak da değiştirebiliriz
        # birçok blogda/eğitimde replay metodu bu şekilde kodlanır ama buradaki for döngüsü çalışma süresi açısından efektif değildir, biz bi alttaki vektörize halini kullanacağız
        # rakamsal örnek vermek gerekirse, T4 FPU'da, 100 episode için çalışması 9 saat sürerken, vectorized olan 15 dk sürmüştür
        if len(self.memory) < self.batch_size: #agent'ın hafızasında yeterince deneyim yoksa devam etmeyelim, deneyim biriktirmeye devam edelim
            return
        minibatch = random.sample(self.memory,self.batch_size)
        for state, action, reward, next_state, done in minibatch:
            if done:
                target = reward
            else:
                target = reward + self.gamma*np.amax(self.online_model.predict(next_state, verbose=0)[0])
            train_target = self.online_model.predict(state, verbose=0)
            train_target[0][action] = target
            #1 epoch ile eğitilmesinin nedeni, online öğrenme, kararlılık, verimlilik ve deneyim tekrarı tekniklerinin etkinliğini en üst düzeye çıkarmaktır.
            #Bu, modelin yeni bilgilere hızlı bir şekilde uyum sağlamasına, öğrenme sürecini stabilize etmesine, eğitim süresini azaltmasına ve aşırı uyum sağlamasını önlemeye yardımcı olur.
            self.online_model.fit(state,train_target, epochs=1, verbose = 0)

    def replay_vectorized(self):
        # üstteki metodun vektörize halidir
        if len(self.memory) < self.batch_size:
            return
        minibatch = random.sample(self.memory,self.batch_size)
        states, actions, rewards, next_states, dones = zip(*minibatch) # [(1, 2, 3, 4, 5), (10, 20, 30, 40, 50)] gibi bir listeyi [(1, 10,.....), (2, 20,......), (3, 30,.......), (4, 40,.......), (5, 50,......)] haline çeviriyoruz, ki bunları numpy diziye çevirebilelim
        if self.verbose:
          print(f"states before numpy dönüşüm in replay metodu:{states}")
        # numpy array'e çevrielim
        self.states = np.array(states)
        self.actions = np.array(actions)
        self.rewards = np.array(rewards)
        self.next_states = np.array(next_states)
        self.dones = np.array(dones)

        if self.verbose:
          print(f"states before reshape in replay:{self.states}, states.shape:{self.states.shape}")
        self.states = self.states.reshape(self.batch_size, self.state_size)  #  (16,1,4) olan shape'i (16,4) yapıyoruz

        if self.verbose:
          print(f"states after reshape in replay:{self.states},states.shape:{self.states.shape}")

        self.next_states = self.next_states.reshape(self.batch_size, self.state_size)
        self.do_calculation() #normalde bunu ayrı metodu yapmaya gerek yoktu ama üst kısmı başka yerde de kullanacağımız için modüler hale getiriyorum

    def do_calculation(self):
        q_expected = self.online_model.predict(self.states, verbose=0) #mevcut statelerden Q-valueları tahminleme yapalım, Q(s,a)
        if self.verbose:
          print(f"q_expected:{q_expected}")

        q_next_target = self.online_model.predict(self.next_states, verbose=0)  # next statelerden Q(s',a') tahminleyelim.
        if self.verbose:
          print(f"q_next:{q_next_target}")

        max_next_q = np.max(q_next_target, axis=1) # q_next_target içindeki en yüksek Q-value'larını alalım
        if self.verbose:
          print(f"max_next_q:{max_next_q}")

        q_targets = self.rewards + self.gamma * max_next_q * (1 - self.dones) #target Q-values hesabı
        if self.verbose:
          print(f"q_targets:{q_targets}")

        # Hangi aksiyon seçildiyse q_expectedda bu aksiyonu targetta ki ile update edelim
        # for idx, action in enumerate(actions):
        #     q_expected[idx][action] = q_targets[idx]
        q_expected[np.arange(self.batch_size), self.actions] = q_targets #vectorized olarak
        if self.verbose:
          print(f"q_expected after update:{q_expected}")

        # eğitelim
        self.online_model.fit(self.states, q_expected, epochs=1, verbose=0)
        self.verbose = False #sadece bir kez bastırmak yeterli bence. Tümünü bastırmak isteyen commentleyebilir


Şimdi agentımızı yaratıp eğitelim.

In [ ]:
env = gym.make("CartPole-v1") #env'i yaratalım
dqlagent = DQLAgent(env) #agent objesini yaratalım, init metodu tetiklenecek
# agent.verbose = True #Uncomment yaparak repaly metodu içindeki print'li kısımların görünmesini sağlayabilirsiniz
printInsideTraining = False # T/F yaparak bu hücre içindeki printlerin görünüp görünmemesini sağlayabilirsiniz
# kodda kalabalık odluğunu düşünüyorsanız ve print'lere gerek duymuyorsanız o satırları silebilirsiniz

train_model(env, dqlagent, episodes=200, print_episode=False) #ilk 100 civarını hızlı öğreniyor ama yeterli öğrenme olmadığı için hızlı oluyr, sonra yavaş yaval güzel öğreniyor, daha uzun sürüyor

110-120'den sonra güzel öğrenmiş gibi. yüksek skorlara ulaşıp sonra tekrar düşmesinin en büyük sebebi, ar ara keşif yapmaya devam etmesi olarak düşünülebilir.

In [ ]:
#agent'ımızı sonraki kullanımları için kaydedelim
dqlagent.save("dqlagent.pkl")

In [ ]:
# test
num_eval_episodes = 10
env_test = gym.make("CartPole-v1", render_mode="rgb_array")
env_test = RecordVideo(env_test, video_folder="cartpole-agent", name_prefix="dql-test",
                  episode_trigger=lambda x: True)
env_test = RecordEpisodeStatistics(env_test, buffer_length=num_eval_episodes)
trained_agent = pickle.load(open("dqlagent.pkl", "rb"))
test_model(env_test, trained_agent, n=num_eval_episodes)

In [ ]:
show_video('cartpole-agent/dql-test-episode-0.mp4')

## DQN

Şimdi DQNAgent adında bir class yaratacağız. Bu, birçok açıdan DQLAgent'a benzeyecek, herşeyi tekrar tekrar kodlamak yerine DQLAgent'ı inherit eden bir class yaratalım ve sadece replay metodunu override edelim.

In [ ]:
class DQNAgent(DQLAgent):
    def __init__(self, env, loss="mse", batch_size=16, learning_rate=0.001, memory_size=1000, target_update=50):
        super().__init__(env, loss, batch_size, learning_rate, memory_size)
        #DQLAgent'a ek olarak şunları yapalım

        # modelleri oluşturalım
        self.online_model   = self.build_model(loss)
        self.target_model   = self.build_model(loss)
        self.target_model.set_weights(self.online_model.get_weights())  # Başlangıçta ağırlıkları eşitleyelim
        self.target_update  = target_update #her 50 updatede güncelleme oalcak, tune edilebilir. çok yüksek seçilirse, hiç update edemeden oyun sonlanabilir, küçük sayılar seçilmeli
        self.update_counter = 0


    def do_calculation(self):
        # 1) Target Q’ları hesapla (target newotrk üzerinden)
        q_next_target = self.target_model.predict(self.next_states, verbose=0)   # Target ağdan next Q’ları al
        max_next_q    = np.max(q_next_target, axis=1)                       # en büyük değeri seç
        q_targets      = self.rewards + self.gamma * max_next_q * (1 - self.dones)
        if self.verbose:
          print(f"q_next_target:{q_next_target}")

        self.continue_calculation(q_targets) #bu kısmı aynen double DQN'de de kullanacağımız için ayırdık, sadece üst kısım değişecek.


    def continue_calculation(self,q_targets):
        # 2) Online ağdan mevcut Q tahminlerini al
        q_expected = self.online_model.predict(self.states, verbose=0)

        # 3) Sadece seçilen aksiyonun Q’sunu güncelle
        q_expected[np.arange(self.batch_size), self.actions] = q_targets

        # 4) Bu batch’ten öğren
        self.online_model.fit(self.states, q_expected, epochs=1, verbose=0)

        # 5) Target ağı periyodik güncelle
        self.update_counter += 1
        if self.update_counter % self.target_update == 0:
            self.target_model.set_weights(self.online_model.get_weights())
            if self.verbose:
              print(f"Updating target network at step {self.update_counter}")

        self.verbose = False #sadece bir kez bastırmak yeterli bence. Tümünü bastırmak isteyen commentleyebilir



In [ ]:
env = gym.make("CartPole-v1")
dqnagent = DQNAgent(env) #DQL'den farklı olarak sadece burayı değiştriyirouz
# agent.verbose = True #Uncomment yaparak repaly metodu içindeki print'li kısımların görünmesini sağlayabilirsiniz
printInsideTraining = False # T/F yaparak bu hücre içindeki printlerin görünüp görünmemesini sağlayabilirsiniz
# kodda kalabalık odluğunu düşünüyorsanız ve print'lere gerek duymuyorsanız o satırları silebilirsiniz

train_model(env, dqnagent,episodes=200, print_episode=False) #ilk 100 civarında çok hızlı ilerledi, 15 dk gibi, sonra hızlı öğrenmeye başladı ve eğitim süresi uzadı

In [ ]:
dqnagent.save("dqnagent.pkl")

In [ ]:
# test
num_eval_episodes = 10
env_test = gym.make("CartPole-v1", render_mode="rgb_array")
env_test = RecordVideo(env_test, video_folder="cartpole-agent", name_prefix="dqn-test",
                  episode_trigger=lambda x: True)
env_test = RecordEpisodeStatistics(env_test, buffer_length=num_eval_episodes)
trained_agent = pickle.load(open("dqnagent.pkl", "rb"))

In [ ]:
test_model(env_test, trained_agent, n=num_eval_episodes)

In [ ]:
show_video('cartpole-agent/dqn-test-episode-0.mp4')

### Hyperparameter tuning

Tüm hyperparamtreleri değil de sadece birkaçı için deneme yapacağım, Bunların dışında
- loss olarak huber denedim ve başarsı düşük çıktığı için burada ele almıyorum
- Belki bir de optimizasyon yöntemi olarak farklı bişey denenebilir
- Activation olarak da tanh ve leaky relu da denenebilir.

ancak eğitim süresini çok daha fazla artırmak istemediğim için bunları yapmıyorum.

In [ ]:
from sklearn.model_selection import ParameterGrid
param_grid = {'update': [5,10,20,40,80],
              "batch_size":[32,64],
              "lr":[0.0001,0.005],
              "memory_size":[2000,5000,10000]
              }

In [ ]:
for e,params in enumerate(ParameterGrid(param_grid)):
  print(params)
  env = gym.make("CartPole-v1")
  dqnagent = DQNAgent(env,"mse",params["batch_size"],params["lr"],params["memory_size"],params["update"])
  printInsideTraining = False # T/F yaparak bu hücre içindeki printlerin görünüp görünmemesini sağlayabilirsiniz
  train_model(env, dqnagent, episodes=150, print_episode=False) #ilk 100 civarında çok hızlı ilerledi, 15 dk gibi, sonra hızlı öğrenmeye başladı ve eğitim süresi uzadı
  dqnagent.save(f"dqnagent_param_{e}.pkl")

In [ ]:
# test
num_eval_episodes = 10
env_test = gym.make("CartPole-v1", render_mode="rgb_array")
env_test = RecordVideo(env_test, video_folder="cartpole-agent", name_prefix="dqn-test-bestparam",
                  episode_trigger=lambda x: True)
env_test = RecordEpisodeStatistics(env_test, buffer_length=num_eval_episodes)
trained_agent = pickle.load(open("dqnagent_param_16.pkl", "rb"))
test_model(env_test, trained_agent, n=num_eval_episodes)

In [ ]:
show_video('cartpole-agent/dqn-test-bestparam-episode-3.mp4')

In [ ]:
save_as_gif("cartpole-agent/dqn-test-bestparam-episode-3.mp4")

## Double DQN

Yine benzer mantıkla bunu da DQNAgent'tan inherit edeceğiz ve sadece do_calculation metodunu override edeceğiz.

In [ ]:
class DoubleDQNAgent(DQNAgent):
    #sadece do_calculation'ı override etmemiz yeterli
    def do_calculation(self):
        # 1) Target Q’ları hesapla
        q_next_target = self.target_model.predict(self.next_states, verbose=0)
        q_online = self.online_model.predict(self.next_states, verbose=0) #aksiyon seçimi için online ağdan tahmin alıyoruz
        next_actions = np.argmax(q_online, axis=1)
        max_next_q = q_next_target[np.arange(self.batch_size), next_actions] #online ağdan en büyük aksiyon seçilir
        q_targets = self.rewards + self.gamma * max_next_q * (1 - self.dones)
        if self.verbose:
          print(f"q_next_target:{q_next_target}")

        self.continue_calculation(q_targets)

In [ ]:
env = gym.make("CartPole-v1")
double_dqnagent = DoubleDQNAgent(env,batch_size=32, learning_rate= 0.005, memory_size= 2000, target_update=5) #hyperparameter tuningden dönen en iyi değerleri kullanalım,
# agent.verbose = True #Uncomment yaparak repaly metodu içindeki print'li kısımların görünmesini sağlayabilirsiniz
printInsideTraining = False # T/F yaparak bu hücre içindeki printlerin görünüp görünmemesini sağlayabilirsiniz
# kodda kalabalık odluğunu düşünüyorsanız ve print'lere gerek duymuyorsanız o satırları silebilirsiniz

train_model(env, double_dqnagent, episodes=200, print_episode=True) #ilk 100 civarında çok hızlı ilerledi, 15 dk gibi, sonra hızlı öğrenmeye başladı ve eğitim süresi uzadı

# Pure tensorflow yaklaşımı

Bizim yazdığımız kodda replay metodu numpy ile vektörize hale getirilerek hızlandırılmıştı. Bunun daha da hızlı yolu tensorlarla çalışmak. Bunun için pure python ile yazılmış gymnasium environment'ının tensorflowcasını kullanmak gerekiyor, yani gymanisum environmentları uygun tf wrapperlarla tensorflowlaştırılıyor.

Bunu biz burada kodlamak yerine işi üstadlara bırakalım, [buradan](https://www.tensorflow.org/agents/tutorials/1_dqn_tutorial) erişebilirsiniz.